# Notebook 06: Modelloptimierung & Overfitting-Reduktion

In diesem Notebook werden verschiedene Strategien zur Verbesserung der Modellgeneralisation umgesetzt. Dazu gehören Feature-Selektion basierend auf Importance, Hyperparameter-Tuning des Random Forest mittels RandomizedSearchCV, ein Vergleich mit einem Gradient-Boosting-Modell sowie optionales Ensembling. Ziel ist die Reduktion von Overfitting und die Steigerung der Vorhersagegenauigkeit.

## 1. Setup & Dateneinlese
In diesem Kapitel werden die benoetigten Bibliotheken importiert, Konstanten definiert und der bereinigte Datensatz eingelesen.

In [33]:
# 1.1 Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import joblib

# 1.2 Konstanten
RANDOM_STATE = 42
DATA_PATH = "../data/processed/cars_features_no_mmr_reduced.csv"

# 1.3 Einlesen des bereinigten Datensatzes
df = pd.read_csv(DATA_PATH)

# 1.4 Erster Überblick
print(f"Shape des DataFrames: {df.shape}")
display(df.head())


Shape des DataFrames: (91562, 22)


,year,condition,odometer,sale_year,sale_month,sale_day,sale_weekday,body,transmission,color,...,season,has_sport,has_limited,has_lx,has_se,has_touring,has_premium,miles_per_year,color_popularity,sellingprice
0,2015,2.0,5559.0,2015,1,13,1,Sedan,automatic,white,...,Winter,0,0,0,1,0,0,5559.0,4,12000.0
1,2012,35.0,45035.0,2014,12,18,3,SUV,automatic,gray,...,Winter,0,1,0,0,0,0,22517.5,3,14100.0
2,2012,46.0,20035.0,2014,12,18,3,SUV,automatic,gray,...,Winter,0,0,0,1,0,0,10017.5,3,20800.0
3,2012,46.0,41115.0,2014,12,18,3,SUV,automatic,white,...,Winter,0,0,0,1,0,0,20557.5,4,22100.0
4,2012,3.0,26747.0,2014,12,17,2,Hatchback,automatic,red,...,Winter,0,0,0,0,0,0,13373.5,6,14000.0


## 2. Preprocessing-Pipeline übernehmen
In diesem Schritt werden die numerischen und kategorialen Merkmale identifiziert und die Vorverarbeitungs-Pipeline aus den vorherigen Notebooks wiederverwendet.


In [34]:
# 2.1 Numerische und kategoriale Features identifizieren
numeric_features = df.select_dtypes(include=[np.number]).drop("sellingprice", axis=1).columns.tolist()
categorical_features = df.select_dtypes(include=["object"]).columns.tolist()

print("Numerische Features:", numeric_features)
print("Kategoriale Features:", categorical_features)

# 2.2 Pipelines definieren
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# 2.3 ColumnTransformer erstellen
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("cat", categorical_pipeline, categorical_features)
])


Numerische Features: ['year', 'condition', 'odometer', 'sale_year', 'sale_month', 'sale_day', 'sale_weekday', 'avg_price_state', 'has_sport', 'has_limited', 'has_lx', 'has_se', 'has_touring', 'has_premium', 'miles_per_year', 'color_popularity']
Kategoriale Features: ['body', 'transmission', 'color', 'interior', 'season']


### 3. Hyperparameter-Tuning für Random Forest (ressourcenschonend)
Dieser Abschnitt führt das Tuning auf einer Stichprobe und mit reduzierter CV- und Suchkonfiguration durch, um Laufzeit zu verkürzen.

In [35]:
# 3.1 Train-/Test-Split (falls noch nicht erfolgt)
from sklearn.model_selection import train_test_split
X = df.drop("sellingprice", axis=1)
y = df["sellingprice"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# 3.2 Stichprobe aus dem Trainingsset entnehmen (z.B. 30%)
X_tune, _, y_tune, _ = train_test_split(
    X_train, y_train, train_size=0.3, random_state=RANDOM_STATE
)

# 3.3 Pipeline mit Random Forest
from sklearn.ensemble import RandomForestRegressor
pipe_rf = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        random_state=RANDOM_STATE,
        n_jobs=1
    ))
])

# 3.4 Parameterbereich festlegen
param_dist = {
    "model__n_estimators": [50, 100, 150],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_leaf": [1, 2],
    "model__max_features": ["sqrt", 0.5]
}

# 3.5 RandomizedSearchCV konfigurieren (cv=3, n_iter=5)
from sklearn.model_selection import RandomizedSearchCV
rs_cv = RandomizedSearchCV(
    estimator=pipe_rf,
    param_distributions=param_dist,
    n_iter=5,                      # nur 5 Kombinationen
    cv=3,                          # 3-fach CV
    scoring="neg_root_mean_squared_error",
    random_state=RANDOM_STATE,
    n_jobs=1                       # single-threaded
)

# 3.6 Tuningsuche auf der Stichprobe ausführen
rs_cv.fit(X_tune, y_tune)

# 3.7 Beste Parameter und Score anzeigen
print("Beste Parameter:", rs_cv.best_params_)
print(f"Bestes CV RMSE (Stichprobe): { -rs_cv.best_score_:.2f}")

Beste Parameter: {'model__n_estimators': 150, 'model__min_samples_leaf': 2, 'model__max_features': 0.5, 'model__max_depth': 20}
Bestes CV RMSE (Stichprobe): 2523.74


## 4. Evaluation des getunten Random Forest
In diesem Abschnitt wird das Modell mit den besten Hyperparametern auf dem gesamten Trainingsset trainiert und auf dem Testset evaluiert.

In [36]:
# Definition der Metrikfunktion (einmalig irgendwo zu Beginn)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def calc_metrics(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    return mae, rmse, r2

# Beispiel: So sieht rs_cv.best_params_ aus und kommt nur aus param_dist
print("Gefundene Parameter:", rs_cv.best_params_)
# z.B.: {'model__n_estimators': 150, 'model__min_samples_leaf': 2, 
#        'model__max_features': 0.5, 'model__max_depth': 20}

# Damit ist sichergestellt, dass nur aus 
# [50, 100, 150], [None, 10, 20], [1, 2], ['sqrt', 0.5] gewählt wird.


Gefundene Parameter: {'model__n_estimators': 150, 'model__min_samples_leaf': 2, 'model__max_features': 0.5, 'model__max_depth': 20}


In [37]:
# 4.1 Pipeline mit den besten Parametern
best_params = rs_cv.best_params_
pipe_rf_tuned = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=best_params["model__n_estimators"],
        max_depth=best_params["model__max_depth"],
        min_samples_leaf=best_params["model__min_samples_leaf"],
        max_features=best_params["model__max_features"],
        random_state=RANDOM_STATE,
        n_jobs=1
    ))
])

# 4.2 Training auf dem vollen Trainingsset
pipe_rf_tuned.fit(X_train, y_train)

# 4.3 Vorhersagen
y_pred_train_tuned = pipe_rf_tuned.predict(X_train)
y_pred_test_tuned  = pipe_rf_tuned.predict(X_test)

# 4.4 Metriken berechnen
mae_tr, rmse_tr, r2_tr = calc_metrics(y_train, y_pred_train_tuned)
mae_te, rmse_te, r2_te = calc_metrics(y_test,  y_pred_test_tuned)

# 4.5 Ergebnisse ausgeben
print("Random Forest (getunt):")
print(f"  Train -> MAE: {mae_tr:.2f}, RMSE: {rmse_tr:.2f}, R²: {r2_tr:.4f}")
print(f"  Test  -> MAE: {mae_te:.2f}, RMSE: {rmse_te:.2f}, R²: {r2_te:.4f}")


Random Forest (getunt):
  Train -> MAE: 1126.92, RMSE: 1538.00, R²: 0.9033
  Test  -> MAE: 1781.53, RMSE: 2405.00, R²: 0.7639


## 5. Vergleich mit Gradient Boosting (sklearn)
Ein GradientBoostingRegressor wird mit einer moderaten Lernrate und Tiefe aufgesetzt, um Regularisierung und Boosting-Effekte zu nutzen. Das Modell wird per 5-facher Kreuzvalidierung geprüft und anschliessend auf Trainings- und Testset evaluiert.

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Pipeline mit GradientBoostingRegressor
pipe_gbr = Pipeline([
    ("preprocessor", preprocessor),
    ("model", GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        random_state=RANDOM_STATE
    ))
])

# 5-fach CV auf Trainingsdaten
scores_gbr = cross_val_score(
    pipe_gbr,
    X_train,
    y_train,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=1
)
print(f"CV RMSE (GradientBoosting): {(-scores_gbr).mean():.2f} ±{scores_gbr.std():.2f}")

# Training und Evaluation auf Trainings- und Testset
pipe_gbr.fit(X_train, y_train)
y_pred_train_gbr = pipe_gbr.predict(X_train)
y_pred_test_gbr  = pipe_gbr.predict(X_test)

mae_tr_gbr, rmse_tr_gbr, r2_tr_gbr = calc_metrics(y_train, y_pred_train_gbr)
mae_te_gbr, rmse_te_gbr, r2_te_gbr = calc_metrics(y_test,  y_pred_test_gbr)

print("\nGradientBoostingRegressor:")
print(f"  Train -> MAE: {mae_tr_gbr:.2f}, RMSE: {rmse_tr_gbr:.2f}, R²: {r2_tr_gbr:.4f}")
print(f"  Test  -> MAE: {mae_te_gbr:.2f}, RMSE: {rmse_te_gbr:.2f}, R²: {r2_te_gbr:.4f}")

CV RMSE (GradientBoosting): 2539.35 ±17.85

GradientBoostingRegressor:
  Train -> MAE: 1913.80, RMSE: 2527.95, R²: 0.7387
  Test  -> MAE: 1913.43, RMSE: 2531.92, R²: 0.7383


## 6. Erweiterte Feature-Erstellung
In diesem Abschnitt werden verschiedene neue Merkmale abgeleitet, um nichtlineare, zeitliche und relative Effekte abzubilden sowie Ausstattungsvielfalt zu erfassen.

In [39]:
import numpy as np
import pandas as pd

# 6.1 Fahrzeugalter
df["age"] = df["sale_year"] - df["year"]

# 6.2 Zyklische Repräsentation von Monat und Wochentag
df["month_sin"]   = np.sin(2 * np.pi * df["sale_month"] / 12)
df["month_cos"]   = np.cos(2 * np.pi * df["sale_month"] / 12)
df["weekday_sin"] = np.sin(2 * np.pi * df["sale_weekday"] / 7)
df["weekday_cos"] = np.cos(2 * np.pi * df["sale_weekday"] / 7)

# 6.3 Wochenend-Flag
df["is_weekend"] = (df["sale_weekday"] >= 5).astype(int)

# 6.4 Anzahl besonderer Ausstattungsmerkmale
trim_flags = ["has_sport", "has_limited", "has_lx", "has_se", "has_touring", "has_premium"]
df["special_count"] = df[trim_flags].sum(axis=1)

# 6.5 Interaktion Zustand × Laufleistung
df["cond_x_mpy"] = df["condition"] * df["miles_per_year"]

# 6.6 Log-Transformationen
df["log_odometer"]     = np.log1p(df["odometer"])

# 6.7 Länge der Kategorienschlüssel
df["body_len"]     = df["body"].str.len()
df["color_len"]    = df["color"].str.len()
df["interior_len"] = df["interior"].str.len()

# 6.8 Quartals-Dummy
df["sale_quarter"] = ((df["sale_month"] - 1) // 3 + 1).astype(int)

# 6.9 Binning von odometer – korrigierte Bin-Grenzen
max_odo = df["odometer"].max()
odo_bins = [0, 20000, 50000, 100000, max_odo]
df["odo_bin"] = pd.cut(
    df["odometer"],
    bins=odo_bins,
    labels=False,
    include_lowest=True
)

# 6.10 Binning von age mit Duplikate-Entfernung
df["age_bin"] = pd.qcut(
    df["age"],
    q=5,
    labels=False,
    duplicates="drop"
)

print("Erweiterte Features hinzugefügt. Aktuelle Spaltenzahl:", df.shape[1])
print("Odometer-Bins:", odo_bins)
print("Eindeutige age_bin-Werte:", sorted(df["age_bin"].dropna().unique()))

Erweiterte Features hinzugefügt. Aktuelle Spaltenzahl: 37
Odometer-Bins: [0, 20000, 50000, 100000, np.float64(170648.0)]
Eindeutige age_bin-Werte: [np.int64(0), np.int64(1), np.int64(2), np.int64(3)]


### 7. Random Forest: Re-Training mit erweiterten Features
Nach der Aktualisierung der Pipeline und des neuen Train-/Test-Splits wird der Random Forest mit den getunten Hyperparametern erneut trainiert und evaluiert.

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 7.1 Neue Feature-Listen
numeric_features     = df.select_dtypes(include=[np.number]).drop("sellingprice", axis=1).columns.tolist()
categorical_features = df.select_dtypes(include=["object"]).columns.tolist()

# 7.2 Aktualisierte Preprocessing-Pipeline
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("cat", categorical_pipeline, categorical_features)
])

# 7.3 Neuer Train-/Test-Split
X = df.drop("sellingprice", axis=1)
y = df["sellingprice"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# 7.4 Pipeline mit getunten RF-Hyperparametern
pipe_rf_final = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=best_params["model__n_estimators"],
        max_depth=best_params["model__max_depth"],
        min_samples_leaf=best_params["model__min_samples_leaf"],
        max_features=best_params["model__max_features"],
        random_state=RANDOM_STATE,
        n_jobs=1
    ))
])

# 7.5 Training
pipe_rf_final.fit(X_train, y_train)

# 7.6 Vorhersagen
y_pred_train = pipe_rf_final.predict(X_train)
y_pred_test  = pipe_rf_final.predict(X_test)

# 7.7 Evaluation
mae_tr, rmse_tr, r2_tr = calc_metrics(y_train, y_pred_train)
mae_te, rmse_te, r2_te = calc_metrics(y_test,  y_pred_test)

# 7.8 Ergebnisse ausgeben
print("Random Forest mit erweiterten Features:")
print(f"  Train -> MAE: {mae_tr:.2f}, RMSE: {rmse_tr:.2f}, R²: {r2_tr:.4f}")
print(f"  Test  -> MAE: {mae_te:.2f}, RMSE: {rmse_te:.2f}, R²: {r2_te:.4f}")

Random Forest mit erweiterten Features:
  Train -> MAE: 1096.89, RMSE: 1498.45, R²: 0.9082
  Test  -> MAE: 1788.88, RMSE: 2414.38, R²: 0.7620


## Vergleich der RF-Varianten ohne MMR

| Version                                | Train RMSE | Test RMSE | Test R²  |
|----------------------------------------|------------|-----------|----------|
| **1. Ungetunt**                        | 912.90     | 2 441.23  | 0.7567   |
| **2. Getunt (Hyperparameter-Optimierung)** | 1 538.00   | 2 405.00  | 0.7639   |
| **3. Getunt + erweiterte Features**    | 1 498.45   | 2 414.38  | 0.7620   |

**Bestes Modell nach MMR-Entfernung:**  
Die **Hyperparameter-getunte Variante** (Version 2) erzielt mit **Test-RMSE ≈ 2 405** und **Test-R² ≈ 0.7639** die besten Werte. Sie reduziert sowohl den Test-RMSE als auch den Overfitting-Gap im Vergleich zum ungetunten Modell und bleibt zugleich genauer als die erweiterte-Feature-Version.  
